# Documentation Neuronal Network



In [ ]:

# We will use this cell to import all the packages you will need in the following - think of it as turning on all your systems
# in your cockpit

# This makes sure that if you change code in your external scripts, they will be updated

from IPython.display import display, clear_output
import numpy as np
import time
import math
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from tensorflow import keras
from keras.api._v2.keras import layers
import tensorflow as tf
import os
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import datetime
import umap

cwd = os.getcwd ()

# now go ahead and Run the cell. This might take a while...
# while the cell is running, you will see ln[*] next to it. Once it finished, you will see the number of execution
# In case you want to interrupt the run of a cell, press Ctrl + C (on your german keyboard, this is Strg + C) 


## Loading the MNIST Dataset

To build and train our model we need to load the MNIST dataset. 

In [ ]:
# Loading the MNIST dataset in one line
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Printing the shape
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

## Defining our function to calculate the mse of two pictures

With this function we want to calculate the mse of two labels to get the most similar ones


In [4]:
def calculate_mse(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err

# Defining our function to search the best images

The function calculates the mse of all images of one number. 
We get the 10 best representing labels of the dataset to train our model with only 10 labels.  

In [ ]:
'''keep_numbers=[]
mse_numbers = []
def searchBestImage():
  for number in range (0,10):
    mse_number = 0.0
    filter_for_number = np.where(y_train == number)[0]
    for number1 in range (len(filter_for_number)):
      for number2 in range (len(filter_for_number)):
        mse_number += calculate_mse(x_train[filter_for_number[number1]],x_train[filter_for_number[number2]])
      mse_number = mse_number / len(filter_for_number)
      mse_numbers.append(mse_number)
    min = np.argmin(mse_numbers)
    keep_index.append(filter_for_number[min])
    mse_numbers.clear()
    filter_for_number = np.where(y_train == number )[0]


x_train = x_train[0:5000]
y_train = y_train[0:5000]

def search_for_most_similar_input(list, index):
  store_mse = 1.0
  remember_index = 1
  for i in range (index, len(list)):
    if i != index: 
      actual_mse = calculate_mse(list[index], list[i])
      if actual_mse < store_mse:
        store_mse = actual_mse
        remember_index = i
  return remember_index
for calc in range (0,5):
  for i in range (0,10):
    filter_for_number = np.where(y_train == i)[0]
    for y in range (len(x_train[filter_for_number])//2):
      index = search_for_most_similar_input(x_train[filter_for_number], y)
      x_train = np.delete(x_train, filter_for_number[index], 0)
      y_train = np.delete(y_train,filter_for_number[index])
      filter_for_number = np.where(y_train == i )[0]
'''
keep_index = [39306, 45926, 35779, 9059, 3639, 29049, 15459, 26151, 56284, 1733]

# Reshaping the training data

Before we can start defining and training our model we need to prepare our training dataset. 
Therefore we need to reshape the training and validation data to fit our input layer. 

In [ ]:
x_train = x_train/255
x_test = x_test/255

# in the next step, we also need to reshape our input to fit our input layer later on. 
# This is due to keras expecting a definition for how many channels your input sample has, as we 
# deal with gray scale this is 1.

x_train= x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
'''
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape: ", x_train.shape)
print("y_train shape: ", y_train.shape)
print("x_test shape", x_test.shape)
print("y_test shape", y_test.shape)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)'''

# Defining an ImageDataGenerator

In [7]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

# Defining UMAP Model

To get better results and predictions of our model we use the umap Model and transform our training data. 

In [ ]:
n_components=10
umap_model = umap.UMAP(n_components=n_components )
x_train_reduced = umap_model.fit_transform(x_train)
x_test_reduced = umap_model.transform(x_test)

# Print out the training data 

Before training we want to have a look at the training data and the shape of the dataset. 

In [15]:
x_train_reduced=x_train_reduced[keep_index]
y_train=y_train[keep_index]
print("y_train:",y_train)
print('x_red:', x_train.shape)

y_train: [0 1 2 3 4 5 6 7 8 9]
x_red: (10, 784)


# Defining our Model

Now we can define our model.

In [ ]:
marvin = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(n_components,)),
  tf.keras.layers.Dense(256, activation='sigmoid',kernel_initializer='glorot_uniform'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(10, activation='softmax',kernel_initializer='glorot_uniform', use_bias=True)
])
marvin.summary()

In [19]:
marvin.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss= 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Defining model parameters and early stopping. 

We define early stopping, the reduce learnrate and the modelcheckpoints to use them later while training. 

In [20]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

#%%
early_stopping = tf.keras.callbacks.EarlyStopping (monitor='val_accuracy',patience=15,mode="max") 
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau (monitor ='val_accuracy', factor =0.5 , min_delta=0.00005 , 
                                                  patience=6, min_lr=0.0001, mode="max")
model_checkpoint = tf.keras.callbacks.ModelCheckpoint (filepath=os.path.join(cwd,'models/DNN_UMAP_30.h5'),
                                                       monitor='val_accuracy',save_best_only=True, mode="max")
#%%

# Fit Model

We train the model with our training dataset and the model parameters we defined. 

In [ ]:
marvin.fit(
    x_train_reduced,
    y_train,
    epochs= 200,
    batch_size= 32,
    validation_data=(x_test_reduced, y_test),
    callbacks=[model_checkpoint,reduce_lr]
)

# Safe Model

In [ ]:
#SAVE MODEL 
model_name = 'model_1'
marvin.save(model_name, save_format='h5')

print('Success! You saved Marvin as: ', model_name)

# Evaluate Model

In [ ]:
#%% EVALUATE MODEL (just to be sure the last val acc value was right)
marvin_reloaded = tf.keras.models.load_model('models/DNN_UMAP_30.h5')
marvin_reloaded.summary()
loss_and_metrics = marvin_reloaded.evaluate(x_test_reduced, y_test, verbose=2)

print("Test Loss", loss_and_metrics[0])
print("Test Accuracy", loss_and_metrics[1])

#%%

# Let Marvin predict on the test set, so we have some data to evaluate his performance.
predictions = marvin_reloaded.predict([x_test_reduced])

# Remember that the prediction of Marvin is a probability distribution over all ten-digit classes
# We want him to assign the digit class with the highest probability to the sample.
predictions = np.argmax(predictions, axis=1)
#pd.DataFrame(predictions)